In [1]:
%display latex

In [2]:
import numpy as np

In [3]:
N = 3
F = GF(2^3, 'x')
x = F.gen()

In [25]:
# additive group character
chi = lambda k: np.exp(np.pi * 1j * int(k.trace()))

def toInt(k):
    return list(F).index(k)

In [26]:
def Proj(u, v=None):
    if not v:
        v = u
    return np.outer(u, v.conj().T)

Id = np.eye(2^N)

def Fourier():
    s = np.zeros((2^N, 2^N), dtype='complex128')
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / np.sqrt(2^N)
    return s
FF = Fourier()

In [27]:
ps = np.load('phase_space.npy')

In [32]:
def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return np.diag([chi(a * k) for k in F])

def X(b):
    return FF.conj().T @ Z(b) @ FF

def D(a, b):
    return phi(a, b) * (Z(a) @ X(b))

The displacement operators should be trace orthogonal, null trace, unitary and hermititan.

In [49]:
# trace orthogonal
for a in F:
    for b in F:
        D1 = D(a, b)
        for c in F:
            for d in F:
                D2 = D(c, d)
                tr = (D1 @ D2.conj().T).trace()
                dl = int(a == c) * int(b == d)
                if not np.isclose(tr, (2^N) * dl):
                    raise Exception(tr, dl, a, b, c, d)

In [50]:
# null trace
for a in F:
    for b in F:
        if a != 0 and b != 0:
            d = D(a, b)
            if not np.isclose(d.trace(), 0):
                raise Exception(a, b)

In [53]:
# hermitian
for a in F:
    for b in F:
        d = D(a, b)
        if not np.allclose(d, d.conj().T):
            raise Exception(a,b)

In [54]:
# unitarity
for a in F:
    for b in F:
        d = D(a,b)
        if not np.allclose(d @ d.conj().T, Id):
            raise Exception

In [58]:
points = [(F(0),F(0)), (x^4,F(0)), (x^4,x^5), (x^3,x^7), (x^3,x^4), (x^6,x^4), (x^6,x^7), (F(0),x^5)]
points

[(0, 0),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (0, x^2 + x + 1)]

In [61]:
op = np.zeros((2^N, 2^N), dtype='complex128')
for j, tau in enumerate(F):
    op += D(*points[j])
eig_state = op / 2^N

In [62]:
def Wigner(alpha, beta):
    wigner_kernel = np.zeros((2^N, 2^N), dtype='complex128')
    for j, ga in enumerate(F):
        for k, de in enumerate(F):
            char = chi(ga * beta + de * alpha)
            wigner_kernel += char * D(ga, de) / 2^N
    return wigner_kernel

In [96]:
def WignerMatrix(state):
    m = np.zeros((2^N, 2^N), dtype='float64')
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            m[i, j] = np.real((Wigner(a, b) @ state).trace())
    return m / 2^N

In [97]:
w = WignerMatrix(eig_state)

In [98]:
np.round(w, 3)

array([[ 0.125, -0.   , -0.   , -0.   , -0.   ,  0.125, -0.   ,  0.   ],
       [ 0.   , -0.   , -0.   ,  0.   ,  0.   , -0.   ,  0.   ,  0.   ],
       [ 0.   , -0.   , -0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.125,  0.   ,  0.   ,  0.125],
       [ 0.125, -0.   , -0.   , -0.   , -0.   ,  0.125, -0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , -0.   , -0.   ,  0.   , -0.   , -0.   ],
       [ 0.   ,  0.   , -0.   , -0.   ,  0.125,  0.   ,  0.   ,  0.125],
       [ 0.   , -0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.   , -0.   ]])

In [99]:
import scipy

In [100]:
mub090 = scipy.io.loadmat('andres/mub090.mat')

In [101]:
mub090.keys()

dict_keys(['__header__', '__version__', '__globals__', 'MUB090'])

In [102]:
mubs = mub090['MUB090']
mubs.shape

(72, 8)

In [134]:
k = 2
s = Proj(mubs[8*k:8*(k+1),0])

In [135]:
np.round(WignerMatrix(s), 3)

array([[-0.   ,  0.   , -0.   ,  0.125, -0.   , -0.   , -0.   ,  0.   ],
       [-0.   , -0.   ,  0.   ,  0.125, -0.   ,  0.   ,  0.   , -0.   ],
       [ 0.125,  0.   , -0.   , -0.   ,  0.   , -0.   , -0.   , -0.   ],
       [ 0.125,  0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.   , -0.   ],
       [ 0.125, -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ],
       [ 0.   , -0.   , -0.   ,  0.125, -0.   , -0.   ,  0.   ,  0.   ],
       [ 0.   , -0.   ,  0.   ,  0.125, -0.   , -0.   , -0.   , -0.   ],
       [ 0.125, -0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.   , -0.   ]])

For $k = 2$ (third MUB indexing from 0) we obtain delta functions.